# Feature Engineering — Extended

This notebook starts from Estelle's starter and adds several extra engineered features (tenure, consumption ratios, price aggregates and volatilities, price-change features). It expects `clean_data_after_eda.csv` and `price_data.csv` to be in `../data/` relative to this notebook. The notebook will save the final dataset to `../data/clean_data_with_features.csv`.


In [2]:
# 1. Import packages
import pandas as pd
import numpy as np
from pathlib import Path


In [3]:
# 2. Load data (adjust path if your notebook sits elsewhere)
clean_path = '../data/clean_data_after_eda.csv'
price_path = '../data/price_data.csv'

# defensive checks
for p in [clean_path, price_path]:
    if not Path(p).exists():
        raise FileNotFoundError(f"Required file not found: {p}. Please ensure the files are in ../data/ relative to this notebook.")

df = pd.read_csv(clean_path)
price_df = pd.read_csv(price_path)

# Parse dates safely where present
for c in ['date_activ','date_end','date_modif_prod','date_renewal']:
    if c in df.columns:
        df[c] = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)

if 'price_date' in price_df.columns:
    price_df['price_date'] = pd.to_datetime(price_df['price_date'], errors='coerce', infer_datetime_format=True)

print('Loaded: df shape =', df.shape, '; price_df shape =', price_df.shape)
display(df.head(2))
display(price_df.head(2))


Loaded: df shape = (14606, 44) ; price_df shape = (193002, 8)


C:\Users\Sandeep\AppData\Local\Temp\ipykernel_11880\1576427450.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[c] = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_11880\1576427450.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[c] = pd.to_datetime(df[c], errors='coerce', infer_datetime_format=True)
C:\Users\Sandeep\AppData\Local\Temp\ipykernel_11880\1576427450.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A str

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,0.000041,0.000908,2.086294,99.530517,44.235794,2.086425,99.530558,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,0.001218,0.000000,0.009482,0.000000,0.000000,0.009485,0.001218,0.000000,0


,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0


In [4]:
# 3.1 Estelle's feature: difference between last and first observed off-peak prices per id
if {'id','price_date'}.issubset(price_df.columns):
    monthly_price_by_id = price_df.groupby(['id','price_date']).agg({
        'price_off_peak_var':'mean',
        'price_off_peak_fix':'mean'
    }).reset_index()

    first_prices = monthly_price_by_id.sort_values(['id','price_date']).groupby('id').first().reset_index()
    last_prices = monthly_price_by_id.sort_values(['id','price_date']).groupby('id').last().reset_index()

    diff = pd.merge(last_prices[['id','price_off_peak_var','price_off_peak_fix']], 
                    first_prices[['id','price_off_peak_var','price_off_peak_fix']], 
                    on='id', suffixes=('_last','_first'))
    diff['offpeak_diff_dec_january_energy'] = diff['price_off_peak_var_last'] - diff['price_off_peak_var_first']
    diff['offpeak_diff_dec_january_power'] = diff['price_off_peak_fix_last'] - diff['price_off_peak_fix_first']
    diff = diff[['id','offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
    display(diff.head())
else:
    print('price_df missing required columns')


,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


In [5]:
# 3.2 Additional features
# Tenure (days)
if 'date_activ' in df.columns and 'date_end' in df.columns:
    df['tenure_days'] = (df['date_end'] - df['date_activ']).dt.days

# Days since last product modification
if 'date_end' in df.columns and 'date_modif_prod' in df.columns:
    df['days_since_modif'] = (df['date_end'] - df['date_modif_prod']).dt.days

# Consumption ratios
if 'cons_last_month' in df.columns and 'cons_12m' in df.columns:
    df['cons_ratio_last_vs_year'] = df['cons_last_month'] / (df['cons_12m'].replace(0, np.nan))
    df['cons_ratio_last_vs_year'] = df['cons_ratio_last_vs_year'].replace([np.inf, -np.inf], np.nan)

# Margin diff
if 'margin_gross_pow_ele' in df.columns and 'margin_net_pow_ele' in df.columns:
    df['margin_diff'] = df['margin_gross_pow_ele'] - df['margin_net_pow_ele']

# ensure id is string
if 'id' in df.columns:
    df['id'] = df['id'].astype(str)
if 'id' in price_df.columns:
    price_df['id'] = price_df['id'].astype(str)

print('New features added to df (sample):')
display(df[['id'] + [c for c in ['tenure_days','days_since_modif','cons_ratio_last_vs_year','margin_diff'] if c in df.columns]].head())


New features added to df (sample):


,id,tenure_days,days_since_modif,cons_ratio_last_vs_year,margin_diff
0,24011ae4ebbe3035111d65fa7c15bc57,1096,227,NaN,0.0
1,d29c2c54acc38ff3c0614d0a653813dd,2566,2566,0.00000,0.0
2,764c75f661154dac3a6c254cd082ea7d,2192,2192,0.00000,0.0
3,bba03439a292a1e166f80264c16191cb,2192,2192,0.00000,0.0
4,149d57cf92fc41cf94415803a877cb4b,2245,2245,0.11887,0.0


In [6]:
# 3.3 Price aggregate features (avg, volatility, spread) per id
price_features = None
if 'id' in price_df.columns:
    gp = price_df.groupby('id')
    price_features = pd.DataFrame({
        'id': gp.size().index.astype(str),
        'avg_offpeak_price_var': gp['price_off_peak_var'].mean().values,
        'std_offpeak_price_var': gp['price_off_peak_var'].std().values,
        'min_offpeak_price_var': gp['price_off_peak_var'].min().values,
        'max_offpeak_price_var': gp['price_off_peak_var'].max().values,
    })
    price_features['spread_offpeak_var'] = price_features['max_offpeak_price_var'] - price_features['min_offpeak_price_var']
    display(price_features.head())
else:
    print('price_df missing id column')


,id,avg_offpeak_price_var,std_offpeak_price_var,min_offpeak_price_var,max_offpeak_price_var,spread_offpeak_var
0,0002203ffbb812588b632b9e628cc38d,0.124338,0.003976,0.119906,0.128067,0.008161
1,0004351ebdd665e6ee664792efc4fd13,0.146426,0.002197,0.143943,0.148405,0.004462
2,0010bcc39e42b3c2131ed2ce55246e3c,0.181558,0.026008,0.150837,0.205742,0.054905
3,0010ee3855fdea87602a5b7aba8e42de,0.118757,0.005049,0.113068,0.123086,0.010018
4,00114d74e963e47177db89bc70108537,0.147926,0.002202,0.145440,0.149902,0.004462


In [7]:
# 3.4 Merge price-level features and Estelle's diff into main df
final = df.copy()
if price_features is not None:
    final = final.merge(price_features, on='id', how='left')
if 'diff' in locals():
    final = final.merge(diff, on='id', how='left')

# Example interaction features
if 'cons_12m' in final.columns and 'avg_offpeak_price_var' in final.columns:
    final['estimated_annual_bill'] = final['cons_12m'] * final['avg_offpeak_price_var']
if 'cons_last_month' in final.columns and 'offpeak_diff_dec_january_energy' in final.columns:
    final['price_sensitivity_proxy'] = final['cons_last_month'] * final['offpeak_diff_dec_january_energy']

# Clean infinities
final = final.replace([np.inf, -np.inf], np.nan)

print('Final shape and sample columns:')
print(final.shape)
display(final.head())


Final shape and sample columns:
(14606, 57)


,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,margin_diff,avg_offpeak_price_var,std_offpeak_price_var,min_offpeak_price_var,max_offpeak_price_var,spread_offpeak_var,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power,estimated_annual_bill,price_sensitivity_proxy
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.0,0.124787,0.007829,0.117479,0.146033,0.028554,0.020057,3.700961,0.000000,0.000000
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.0,0.149609,0.002212,0.146033,0.151367,0.005334,-0.003767,0.177779,697.177552,-0.000000
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.0,0.170512,0.002396,0.167798,0.172468,0.004670,-0.004670,0.177779,92.758347,-0.000000
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,0.0,0.151210,0.002317,0.148586,0.153133,0.004547,-0.004547,0.177779,239.516772,-0.000000
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,0.0,0.124174,0.003847,0.119906,0.128067,0.008161,-0.006192,0.162916,549.471425,-3.256992


In [8]:
# 4. Save final dataset for modeling
out_path = '../data/clean_data_with_features.csv'
final.to_csv(out_path, index=False)
print('Saved final dataset to', out_path)


Saved final dataset to ../data/clean_data_with_features.csv


## Next steps

- Review correlations / feature importance to select top predictors.
- Run simple models (logistic regression, random forest) and evaluate AUC/precision/recall.
- Iterate: create more interaction features or aggregate windows (3-month avg price change, recent price trend slope, etc.).


## Submission Checklist & Feature Summary

**Checklist before submitting**
- [ ] Drop irrelevant columns and document why.
- [ ] Extract date-derived features (year, month, tenure, etc.).
- [ ] Create ratio and interaction features (consumption × price, margin per power).
- [ ] Compute price aggregates (avg, std, spread) and price-change features.
- [ ] Merge and save final dataset as `../data/clean_data_with_features.csv`.
- [ ] Include a short feature summary table (original count, dropped, added, final count).

Below is an automated **Feature Summary** cell that will compare the original `clean_data_after_eda.csv` columns with the final saved `clean_data_with_features.csv` and produce a small table you can include in your submission. Run that cell after you've executed the notebook so both files exist in `../data/`.


In [9]:
# Feature summary: compare original and final feature sets
import pandas as pd
from pathlib import Path

orig_path = Path('../data/clean_data_after_eda.csv')
final_path = Path('../data/clean_data_with_features.csv')

if not orig_path.exists():
    print(f"Original data not found at {orig_path}. Please run the EDA/save step to create it.")
else:
    orig = pd.read_csv(orig_path)
    orig_cols = list(orig.columns)

if not final_path.exists():
    print(f"Final features file not found at {final_path}. Please run this feature-engineering notebook first to generate it.")
else:
    final = pd.read_csv(final_path)
    final_cols = list(final.columns)

# If both exist, show summary
if orig_path.exists() and final_path.exists():
    dropped = [c for c in orig_cols if c not in final_cols]
    added = [c for c in final_cols if c not in orig_cols]
    summary = pd.DataFrame({
        'metric': ['original_feature_count', 'final_feature_count', 'dropped_count', 'added_count'],
        'value': [len(orig_cols), len(final_cols), len(dropped), len(added)]
    })
    display(summary)
    print('\nSample dropped columns (up to 10):', dropped[:10])
    print('\nSample added columns (up to 20):', added[:20])
    # also provide a CSV with the list for submission documentation
    docs_path = Path('../data/feature_changes_summary.csv')
    pd.DataFrame({'dropped': dropped + [None]*(max(0, len(added)-len(dropped))),
                  'added': added + [None]*(max(0, len(dropped)-len(added)))}
                ).to_csv(docs_path, index=False)
    print(f'Wrote detailed feature change list to {docs_path}')


,metric,value
0,original_feature_count,44
1,final_feature_count,57
2,dropped_count,0
3,added_count,13



Sample dropped columns (up to 10): []

Sample added columns (up to 20): ['tenure_days', 'days_since_modif', 'cons_ratio_last_vs_year', 'margin_diff', 'avg_offpeak_price_var', 'std_offpeak_price_var', 'min_offpeak_price_var', 'max_offpeak_price_var', 'spread_offpeak_var', 'offpeak_diff_dec_january_energy', 'offpeak_diff_dec_january_power', 'estimated_annual_bill', 'price_sensitivity_proxy']
Wrote detailed feature change list to ..\data\feature_changes_summary.csv
